In [ ]:
from selenium import webdriver
import selenium as se
from selenium.webdriver import ActionChains
# import undetected_chromedriver as uc 
import time
from time import sleep
import pandas as pd 

options = se.webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-infobars')
options.add_argument('--disable-extensions')
options.add_argument('--profile-directory=Default')
options.add_argument('--incognito')
options.add_argument('--disable-plugins-discovery')
options.add_argument('--start-maximized')
driver = webdriver.Chrome(options=options)

g2_info = [] # Save info scrapped from g2
def fetch(URL):
    driver = webdriver.Chrome(options=options)
    driver.get(URL)
    print(driver.title)
    return driver

def hover_click(xpath):
    # Show, hover, then click button
    button = d.find_element_by_xpath(xpath)  
    d.execute_script("arguments[0].scrollIntoView();", button) 
    d.execute_script('window.scrollBy(0,-80)') # scroll up a bit so that the button is truly in view
    ActionChains(d).move_to_element(button).perform() # hover, otherwise can't click
    d.execute_script('arguments[0].click();', button)


def retriever(d, name):
    sleep(0.8)
    URL = d.find_element_by_xpath('/html/head/meta[10]').get_property('content')
    website = d.find_element_by_xpath('//*[@id="leads-sticky-top"]/div/div[1]/div[3]/div[1]/div[2]/div[2]/div[1]/div[1]/div/div/a').get_property('href')

    # try & except on rating and num_view, because some companies don't have them
    try:
        rating = d.find_element_by_xpath('//*[@id="products-dropdown"]/div[1]/div[1]/span[1]').get_attribute('innerHTML')
    except NoSuchElementException:
        rating = None

    try:
        num_review = d.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div[2]/div[2]/div[2]/div/div[2]/span/meta[4]').get_property('content')
    except NoSuchElementException:
        num_review = None

    description = d.find_element_by_xpath('//*[@id="leads-sticky-top"]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/p').get_attribute('innerHTML')

    # Click 'show more' to see details
    hover_click('//*[@id="leads-sticky-top"]/div/div[1]/div[3]/div/div[2]/a')
    detail = d.find_element_by_xpath('//*[@id="leads-sticky-top"]/div/div[1]/div[3]/div[1]/div[2]').get_attribute('innerText')

    pricing_url = 'https://www.g2.com/products/' + name + '/pricing'
    d = fetch(pricing_url)
    try:
        pricing = d.find_element_by_xpath('//*[@id="leads-sticky-top"]/div/div[1]/div[2]/table/tbody').get_attribute('innerText')
    except NoSuchElementException:
        pricing = None
    
    alternative_url = 'https://www.g2.com/products/' + name + '/competitors/alternatives'
    d = fetch(alternative_url)
    show_more_xpath = '/html/body/div[1]/div/div/div[1]/div/div[5]/div[1]/div/div/div[3]/div/div/a'
    # Can't use hover_click(), requires a different method
    button = d.find_element_by_xpath(show_more_xpath)  
    d.execute_script('window.scrollBy(0,220)') # Scroll down a bit
    ActionChains(d).move_to_element(button).perform() 
    d.execute_script('arguments[0].click();', button)
    alternatives = d.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div/div[5]/div[1]/div/div/div[3]/div/ul').get_attribute('innerText')

    g2_info.append([name, URL, website, rating, num_review, description, detail, pricing, alternatives])


df = pd.read_csv('data_scientist_intern_g2_scraper.csv')
companies = df['NAME']
for company in companies:
    product_url = 'https://www.g2.com/products/' + company + '/reviews'
    d = fetch(product_url)
    # If can't find this company(404), ignore this company and continue
    # Else retrieve information
    try:
        d.find_element_by_class_name('error-text-number')
    except NoSuchElementException:
        retriever(d,company)
    else:
        continue
    sleep(0.8)


In [ ]:
# Transfer scrapped data into a dataframe
col = ['NAME', 'URL', 'website', 'rating', 'num_review', 'description', 'detail', 'pricing', 'alternatives']
g2_info = pd.DataFrame(g2_info, columns=col)

# This code is used to remove the 'show less' from the end of column detail and alternatives. Unfortunately I missed the oppotunity to use it.
# g2_info['detail'] = g2_info['detail'].map(lambda x: str(x)[:-9])
# g2_info['alternatives'] = g2_info['alternatives'].map(lambda x: str(x)[:-9])

# Join the original table with new table
scraper = pd.merge(df,g2_info, how='left' ,on=['NAME'])
scraper.to_csv('data_scientist_intern_g2_scraper.csv', index=None)

I only scrapped one company's pricing and alternatives info before CloudFlare blocked me. Here is the result.

In [275]:
g2_info

[['Shift',
  'https://www.g2.com/products/shift/reviews',
  'https://shiftelearning.com/',
  '4.0',
  '2',
  "SHIFT is a web-based e-learning authoring tool that offers a programming- free environment for creating highly interactive e-learning courses that can be published on any standards-compliant LMS, PC's, tablet devices or mobile devices.",
  'SHIFT Details\nWebsite\nSHIFT\nDiscussions\nSHIFT Community\nLanguages Supported\nEnglish, Spanish\nProduct Description\n\nWith SHIFT, anyone can create visually compelling training courses that seamlessly play across phones, tablets, and browsers.\n\nSeller Details\nSeller\nAura Interactiva\nCompany Website\nwww.aurainteractiva.com\nYear Founded\n1996\nTwitter\n@aurainteractiva\n6,424 Twitter followers\nLinkedIn® Page\nwww.linkedin.com\n62 employees on LinkedIn®\nKG\n\t\nOverview Provided by:\nKarla Gutierrez\nMarketing Manager and Digital Strategist at Aura Interactiva\nShow Less',
  None,
  'Brainshark\nSAP Litmos\nLessonly\niSpring Suite